# Photobooth

In [8]:
import cv2
import numpy as np
# pip install Pillow
    # open Anaconda Prompt and paste above line (without '#') to install package
from PIL import Image

In [10]:
# function that gives a range on hues given a color
def get_limits(color):
    c = np.uint8([[color]])
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)
    
    lowerLimit = hsvC[0][0][0] - 10, 100, 100
    upperLimit = hsvC[0][0][0] + 10, 255, 255
    # the +/-10 defines the range of hues that fall within the limits (the h in hsv)
    # the range on saturation and value is much bigger because we are only looking for hue
    
    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [20]:
color = [194, 137, 50]            # color to detect (in BGR colorspace)
line_color = [255, 191, 0]        # color used to draw path
line_color2 = [255, 0, 159]       # secondary color used to draw path
line_size = 4                     # line thickness
line_size2 = 7                    # line thickness of secondary color (should be > line_size to be visible, make < line_size to hide)
countdown_length = 10             # amount of time you have to draw before picture is taken (roughly in seconds)
detection_size = 500              # size of detection required to mark color as an object
capture = cv2.VideoCapture(0)     # picks camera to use (usually 0 or 1)

if (line_color == [0, 0, 0]): # breaks if you use black ([0, 0, 0]), so we adjust it
    line_color = [1, 1, 1]
if (line_color2 == [0, 0, 0]): # breaks if you use black ([0, 0, 0]), so we adjust it
    line_color2 = [1, 1, 1]

while True:
    first = False
    picture = False
    path = 0
    timer_mod = 3
    countdown = countdown_length * timer_mod * 11
    height = 0
    width = 0
    bboxes = list() # array that will hold locations of detected colors
    bboxesPrev = list() # array that will hold the locations of detected colors from the previous frame
    while True:
        ret, frame = capture.read()
        frame = cv2.flip(frame, 1)
        
        if (first == False): # runs only once
            # get height and width of video
            shape = frame.shape
            height = shape[0]
            width = shape[1]
            cqm = width / 640  # scales detection with video quality (because it depends on number of pixels)
            line_thickness = int(line_size * cqm)
            line_thickness2 = int(line_thickness * line_size2/line_size)
            # create path image to draw on (based on shape of webcam feed)
            path = np.array(Image.new("RGB", (width, height), (0,0,0)))
            if line_size2 != 0:
                path2 = np.array(Image.new("RGB", (width, height), (0,0,0)))
            # set first to True so this block doesn't run again
            first = True
        
        frame_blur = cv2.GaussianBlur(frame, (11, 11), 17) # blurring the image may help get the desired result, but it can be removed
        
        frame_hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV) # convert to HSV
        lowerLimit, upperLimit = get_limits(color) # range of hues that we want the software to detect
        
        # define bboxes and bboxesPrev to decide where to draw lines (changes every frame)
        bboxesPrev = bboxes 
        bboxes = list() # empty the array to add new data
    
        # check image for colors and assign bounding boxes
        mask = cv2.inRange(frame_hsv, lowerLimit, upperLimit) # detects objects in color range
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        bboxes = list()
        for cnt in contours:
            if cv2.contourArea(cnt) > detection_size*cqm**2:  # only continues if size of the object is large enough (removes noise)
                x1, y1, w, h = cv2.boundingRect(cnt) # finds a bounding box for each object
                c = list([int(x1 + w/2), int(y1 + h/2)])  # centerpoint of bbox
                # check other bboxes to see if we want to combine them into 1 box
                newBox = list([c[0], c[1], x1, y1, w, h])
                for i in bboxes[:]:
                    cxi, cyi, x1i, y1i, wi, hi = i  # bbox we check the newBox against
                    if np.sqrt((c[0] - cxi)**2 + (c[1] - cyi)**2) < np.sqrt(w**2 + h**2)/3 + np.sqrt(wi**2 + hi**2)/3 + cqm*30: # if centerpoints are close enough (scales with box size)
                        bboxes.remove(i)
                        # reassign bbox boundaries so the new box contains both nearby boxes
                        newBox[2], newBox[3] = min(x1, x1i), min(y1, y1i)  # reassigns x1 and y1 values
                        newBox[4], newBox[5] = max(x1+w, x1i+wi) - newBox[2], max(y1+h, y1i+hi) - newBox[3]  # reassgins w and h values
                        newBox[0], newBox[1] = int(newBox[2] + newBox[4]/2), int(newBox[3] + newBox[5]/2)  # reassigns centerpoint values
                bboxes.append(newBox)
        
        # draw lines between an object's old and new positions
            # compares data from "bboxes" and "bboxesPrev"
        # also draws bounding boxes
        for i in bboxes:
            cxi, cyi, x1i, y1i, wi, hi = i
            cv2.rectangle(frame, (x1i, y1i), (x1i + wi, y1i + hi), (0, 0, 255), 2) # draws bounding boxes for each bbox in the current frame
            for j in bboxesPrev:
                cxj, cyj, x1j, y1j, wj, hj = j
                if np.sqrt((cxj - cxi)**2 + (cyj - cyi)**2) < np.sqrt(wj**2 + hj**2)/3 + np.sqrt(wi**2 + hi**2)/3 + cqm*25:     # will draw a line only if two centerpoints are close enough (scales with box size)
                    if line_size2 != 0:
                        cv2.line(path2, (i[0], i[1]), (j[0], j[1]), line_color2, line_thickness2)
                    cv2.line(path, (i[0], i[1]), (j[0], j[1]), line_color, line_thickness)
    
                    break  # break so two lines don't get drawn from the same point
    
        # overlay the path over the webcam feed
        # for path2: secondary line color
        if line_size2 != 0:
            mask2 = cv2.cvtColor(path2, cv2.COLOR_BGR2GRAY)
            mask2 = cv2.threshold(mask2, 0, 255, cv2.THRESH_BINARY)[1]
            mask2inv = cv2.bitwise_not(mask2)
            pathfg = cv2.bitwise_and(path2, path2, mask=mask2) # extract foreground
            framebg = cv2.bitwise_and(frame, frame, mask=mask2inv) # extract background
            frame = cv2.add(framebg, pathfg) # combine foreground and background
        # for path: main line color
        mask2 = cv2.cvtColor(path, cv2.COLOR_BGR2GRAY)
        mask2 = cv2.threshold(mask2, 0, 255, cv2.THRESH_BINARY)[1]
        mask2inv = cv2.bitwise_not(mask2)
        pathfg = cv2.bitwise_and(path, path, mask=mask2) # extract foreground
        framebg = cv2.bitwise_and(frame, frame, mask=mask2inv) # extract background
        frame = cv2.add(framebg, pathfg) # combine foreground and background
    
        # Countdown
        countdown -= 1
        displayNum = int(countdown / (timer_mod * countdown_length))
        if countdown != 0:
            cv2.putText(frame, f'{displayNum}', (int(8*cqm), int(100*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(4*cqm), (1, 1, 1), int(5*cqm))
            cv2.putText(frame, f'{displayNum}', (int(12*cqm), int(100*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(4*cqm), (1, 1, 1), int(5*cqm))
            cv2.putText(frame, f'{displayNum}', (int(10*cqm), int(98*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(4*cqm), (1, 1, 1), int(5*cqm))
            cv2.putText(frame, f'{displayNum}', (int(10*cqm), int(102*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(4*cqm), (1, 1, 1), int(5*cqm))
            cv2.putText(frame, f'{displayNum}', (int(10*cqm), int(100*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(4*cqm), (100, 100, 100), int(5*cqm))
            cv2.putText(frame, 'click [spacebar] to restart timer', (25, height - 25), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), (100, 100, 100), int(1*cqm))
        if countdown == 0:
            cv2.imwrite('Photobooth Output.jpg', frame)
            picture = True
            break
        
        # display videos
        # cv2.imshow('mask', mask)
        cv2.imshow('photobooth', frame)
    
        # key inputs
        key = cv2.waitKey(10) & 0xFF
        if key == 32:               # 32 is ASCII for [spacebar]
            first = False           # resets countdown and clears drawing
            countdown = countdown_length * timer_mod * 11
        if key == 27:               # 27 is ASCII for [ESC]
            break                   # ends photobooth early

    if key == 27:               # 27 is ASCII for [ESC]
        key = 0
        break                   # HOLD [ESC] to end program
    
    # display image taken for a little bit
    if picture == True:
        im = cv2.imread('Photobooth Output.jpg')
        cv2.imshow('photobooth', im)
        cv2.waitKey(1000)
            
    # display method of picture collection (qr code)
        qr = cv2.imread('QR Code.png')
        cv2.imshow('photobooth', qr)
        while True:
            key = cv2.waitKey(10) & 0xFF
            if key == 32:               # 32 is ASCII for [spacebar]
                break                   # returns to beginning of program
            if key == 27:               # 27 is ASCII for [ESC]
                break                   # ends photobooth early

    if key == 27:               # 27 is ASCII for [ESC]
        key = 0
        break                   # HOLD [ESC] to end program


capture.release()
cv2.destroyAllWindows()  # closes window, only reaches here when [ESC] is pressed

here
here
here
here
